In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

import sys
import os
import time

from PIL import Image

import umap
import pickle
from math import *

from shutil import copyfile

import pygame

DEVICE = "cpu"

def tovar(x):
    return torch.FloatTensor(x).to(DEVICE)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#f = open("interactive/sim1_rules.txt", "w")
"""
for i in range(6000):
    f2 = open("data_2d/%.06d/rule.txt" % i, "r")
    toks = f2.readline().split(' ')
    ruleid = int(toks[3])
    colors = int(toks[0])
    t1colors.append(colors)
    t1rules.append(ruleid)
    f2.close()
    
t1rules = np.array(t1rules)
t1colors = np.array(t1colors)

np.save("interactive/t1rules.npy", t1rules)
np.save("interactive/t1colors.npy", t1colors)

t2rules = []
t2colors = []

colorset = { }
for i in range(1,7):
    rulesize = i * 5**i
    colorset[rulesize] = i

for i in range(1700):
    data = np.load("data3_2d/%.06d/rule.npy" % i).astype(np.uint8)
    t2rules.append(data)
    t2colors.append(colorset[data.shape[0]])

t2rules = np.array(t2rules)
t2colors = np.array(t2colors)

np.save("interactive/t2rules.npy", t2rules)
np.save("interactive/t2colors.npy", t2colors)
"""
"""
colorset = { }
for i in range(1,7):
    rulesize = i * 9**i
    colorset[rulesize] = i

t3rules = []
t3colors = []
for i in range(2000):
    data = np.load("data4_2d/%.06d/rule.npy" % i).astype(np.uint8)
    t3rules.append(data)
    t3colors.append(colorset[data.shape[0]])

t3rules = np.array(t3rules)
t3colors = np.array(t3colors)

np.save("interactive/t3rules.npy", t3rules)
np.save("interactive/t3colors.npy", t3colors)

"""
#    f.write("%d %d\n" % (colors, ruleid))
#f.close()

'\ncolorset = { }\nfor i in range(1,7):\n    rulesize = i * 9**i\n    colorset[rulesize] = i\n\nt3rules = []\nt3colors = []\nfor i in range(2000):\n    data = np.load("data4_2d/%.06d/rule.npy" % i).astype(np.uint8)\n    t3rules.append(data)\n    t3colors.append(colorset[data.shape[0]])\n\nt3rules = np.array(t3rules)\nt3colors = np.array(t3colors)\n\nnp.save("interactive/t3rules.npy", t3rules)\nnp.save("interactive/t3colors.npy", t3colors)\n\n'

In [3]:
t1rules = []
t1colors = []

t1rules = np.load("interactive/t1rules.npy")
t1colors = np.load("interactive/t1colors.npy")
t2rules = np.load("interactive/t2rules.npy", allow_pickle=True)
t2colors = np.load("interactive/t2colors.npy", allow_pickle=True)
t3rules = np.load("interactive/t3rules.npy", allow_pickle=True)
t3colors = np.load("interactive/t3colors.npy", allow_pickle=True)

In [4]:
class Triplet(nn.Module):
    def __init__(self):
        super(Triplet,self).__init__()
        
        self.l1a = nn.Conv3d(3,32,5,padding=2)
        nn.init.orthogonal_(self.l1a.weight, gain=sqrt(2))
        self.b1a = nn.BatchNorm3d(32)
        self.l1b = nn.Conv3d(32,32,5,padding=2)
        self.b1b = nn.BatchNorm3d(32)
        nn.init.orthogonal_(self.l1b.weight, gain=sqrt(2))
        self.p1 = nn.MaxPool3d(2)
        
        self.l2a = nn.Conv3d(32,64,5,padding=2)
        self.b2a = nn.BatchNorm3d(64)
        nn.init.orthogonal_(self.l2a.weight, gain=sqrt(2))
        self.l2b = nn.Conv3d(64,64,5,padding=2)
        self.b2b = nn.BatchNorm3d(64)
        nn.init.orthogonal_(self.l2b.weight, gain=sqrt(2))
        self.p2 = nn.MaxPool3d(2)

        self.l3a = nn.Conv3d(64,128,5,padding=2)
        self.b3a = nn.BatchNorm3d(128)
        nn.init.orthogonal_(self.l3a.weight, gain=sqrt(2))
        self.l3b = nn.Conv3d(128,128,5,padding=2)
        self.b3b = nn.BatchNorm3d(128)
        nn.init.orthogonal_(self.l3b.weight, gain=sqrt(2))
        self.p3 = nn.MaxPool3d(2)
        
        self.l4a = nn.Conv3d(128,256,5,padding=2)
        self.b4a = nn.BatchNorm3d(256)
        nn.init.orthogonal_(self.l4a.weight, gain=sqrt(2))
        self.l4b = nn.Conv3d(256,256,5,padding=2)
        self.b4b = nn.BatchNorm3d(256)
        nn.init.orthogonal_(self.l4b.weight, gain=sqrt(2))
        self.p4 = nn.MaxPool3d(2)
        
        self.l5 = nn.Linear(256,16,1)
        nn.init.orthogonal_(self.l5.weight, gain=sqrt(2))
        
        self.optim = torch.optim.Adam(self.parameters(), lr=1e-4)
        
    def forward(self, x):
        z = x
        z = F.leaky_relu(self.l1a(z))
        z = self.b1a(z)
        z = F.leaky_relu(self.l1b(z))
        z = self.b1b(z)
        z = self.p1(z)
        
        z = F.leaky_relu(self.l2a(z))
        z = self.b2a(z)
        z = F.leaky_relu(self.l2b(z))
        z = self.b2b(z)
        z = self.p2(z)
        
        z = F.leaky_relu(self.l3a(z))
        z = self.b3a(z)
        z = F.leaky_relu(self.l3b(z))
        z = self.b3b(z)
        z = self.p3(z)
        
        z = F.leaky_relu(self.l4a(z))
        z = self.b4a(z)
        z = F.leaky_relu(self.l4b(z))
        z = self.b4b(z)
        z = self.p4(z).mean(4).mean(3).mean(2)
        
        z = self.l5(z)
        z = z/torch.sqrt(torch.sum(z**2,1)+1e-4).unsqueeze(1)
        
        return z

"""
class Totalistic():
    def __init__(self, colors, rule):
        self.rule = rule
        self.colors = colors
        
    def step(self, state):
        result = cpl.evolve2d(state[np.newaxis,:,:], timesteps=2, 
                     apply_rule=lambda n, c, t: cpl.totalistic_rule(n,k=self.colors,rule=self.rule), r=1)
        return result[-1,:,:]
"""

class Totalistic():
    def __init__(self, colors, rule):
        self.colors = colors
        
        transitions = np.zeros(9*(colors-1)+1)
        mult = 1
        
        for i in range(9*(colors-1)+1):
            transitions[i] = (rule//mult)%self.colors
            mult *= self.colors
        # Totalistic:
        self.transitions = torch.LongTensor(transitions).to(DEVICE)
    
    def step(self, state):
        XR = state.shape[0]
        YR = state.shape[1]
        
        self.state = torch.LongTensor(state).to(DEVICE)
        idx = torch.arange(XR*YR).long().to(DEVICE)

        padstate = torch.cat([self.state[-1:,:], self.state, self.state[:1,:]], 0)
        padstate = torch.cat([padstate[:,-1:], padstate, padstate[:,:1]], 1)

        multiplier = self.colors
        histogram = self.state.view(XR*YR)

        shiftstate = padstate[:-2,1:-1].contiguous().view(XR*YR)
        histogram[idx] += shiftstate[idx]
        shiftstate = padstate[2:,1:-1].contiguous().view(XR*YR)
        histogram[idx] += shiftstate[idx]
        shiftstate = padstate[1:-1,:-2].contiguous().view(XR*YR)
        histogram[idx] += shiftstate[idx]
        shiftstate = padstate[1:-1,2:].contiguous().view(XR*YR)
        histogram[idx] += shiftstate[idx]
        shiftstate = padstate[:-2,:-2].contiguous().view(XR*YR)
        histogram[idx] += shiftstate[idx]
        shiftstate = padstate[:-2,2:].contiguous().view(XR*YR)
        histogram[idx] += shiftstate[idx]
        shiftstate = padstate[2:,:-2].contiguous().view(XR*YR)
        histogram[idx] += shiftstate[idx]
        shiftstate = padstate[2:,2:].contiguous().view(XR*YR)
        histogram[idx] += shiftstate[idx]

        result = self.transitions[histogram].view(XR, YR)
        
        return result.cpu().detach().numpy()

class FourNeighbor():
    def __init__(self, colors, rule):
        self.colors = colors
        
        # 4 neighbor outer symmetric
        # 
        # k (center) * 5^k (outer [0,1,2,3,4] for each color)
        self.transitions = torch.LongTensor(rule).to(DEVICE)
    
    def step(self, state):
        XR = state.shape[0]
        YR = state.shape[1]
        
        self.state = torch.LongTensor(state).to(DEVICE)
        idx = torch.arange(XR*YR).long().to(DEVICE)

        padstate = torch.cat([self.state[-1:,:], self.state, self.state[:1,:]], 0)
        padstate = torch.cat([padstate[:,-1:], padstate, padstate[:,:1]], 1)

        multiplier = int(self.colors)
        indices = self.state.view(XR*YR).long()

        histogram = torch.zeros((XR*YR, self.colors)).long().to(DEVICE)

        shiftstate = padstate[:-2,1:-1].contiguous().view(XR*YR)
        histogram[idx, shiftstate[idx]] += 1
        shiftstate = padstate[2:,1:-1].contiguous().view(XR*YR)
        histogram[idx, shiftstate[idx]] += 1
        shiftstate = padstate[1:-1,:-2].contiguous().view(XR*YR)
        histogram[idx, shiftstate[idx]] += 1
        shiftstate = padstate[1:-1,2:].contiguous().view(XR*YR)
        histogram[idx, shiftstate[idx]] += 1

        for k in range(self.colors):
            indices += multiplier * histogram[:,k] 
            multiplier *= 5

        result = self.transitions[indices].view(XR, YR)
        
        return result.cpu().detach().numpy()

class EightNeighbor():
    def __init__(self, colors, rule):
        self.colors = colors
        
        # 8 neighbor outer symmetric
        # 
        # k (center) * 9^k (outer [0,...,8] for each color)
        self.transitions = torch.LongTensor(rule).to(DEVICE)
    
    def step(self, state):
        XR = state.shape[0]
        YR = state.shape[1]
        
        self.state = torch.LongTensor(state).to(DEVICE)
        idx = torch.arange(XR*YR).long().to(DEVICE)

        padstate = torch.cat([self.state[-1:,:], self.state, self.state[:1,:]], 0)
        padstate = torch.cat([padstate[:,-1:], padstate, padstate[:,:1]], 1)

        multiplier = int(self.colors)
        indices = self.state.view(XR*YR).long()

        histogram = torch.zeros((XR*YR, self.colors)).long().to(DEVICE)

        shiftstate = padstate[:-2,1:-1].contiguous().view(XR*YR)
        histogram[idx, shiftstate[idx]] += 1
        shiftstate = padstate[2:,1:-1].contiguous().view(XR*YR)
        histogram[idx, shiftstate[idx]] += 1
        shiftstate = padstate[1:-1,:-2].contiguous().view(XR*YR)
        histogram[idx, shiftstate[idx]] += 1
        shiftstate = padstate[1:-1,2:].contiguous().view(XR*YR)
        histogram[idx, shiftstate[idx]] += 1
            
        shiftstate = padstate[:-2,:-2].contiguous().view(XR*YR)
        histogram[idx, shiftstate[idx]] += 1
        shiftstate = padstate[:-2,2:].contiguous().view(XR*YR)
        histogram[idx, shiftstate[idx]] += 1
        shiftstate = padstate[2:,:-2].contiguous().view(XR*YR)
        histogram[idx, shiftstate[idx]] += 1
        shiftstate = padstate[2:,2:].contiguous().view(XR*YR)
        histogram[idx, shiftstate[idx]] += 1

        for k in range(self.colors):
            indices += multiplier * histogram[:,k] 
            multiplier *= 9

        result = self.transitions[indices].view(XR, YR)
        
        return result.cpu().detach().numpy()

RGB = np.array([
    [0,0,0],
    [1,1,1],
    [1,0,0],
    [0,1,0],
    [0,0,1],
    [1,0,1],
    [1,1,0],
    [0,1,1]])
    
class CA():
    def __init__(self, XR, YR):
        self.XR = XR
        self.YR = YR
        self.engine = None
        
        self.state = np.zeros((XR,YR))
        
    def set_rule(self, ruletype, ruleidx, colors):
        self.colors = colors
        self.ruletype = ruletype
        self.ruleidx = ruleidx
        
        if ruletype == 0:
            self.engine = Totalistic(colors, t1rules[ruleidx])
        elif ruletype == 1:
            self.engine = FourNeighbor(colors, t2rules[ruleidx])
        elif ruletype == 2:
            self.engine = EightNeighbor(colors, t3rules[ruleidx])
            
    def render(self):
        im = RGB[self.state.reshape((self.XR*self.YR))].reshape((self.XR, self.YR, 3))
        return (im*255).astype(np.uint8)
        
    def init_state(self):
        self.state = np.random.randint(self.colors, size=(self.XR, self.YR))
        
    def step(self):
        self.state = self.engine.step(self.state)

In [5]:
import parse

im = pygame.image.load("ca2d_embedding_multi_2.png")

index = { }
datasets = { 'data_2d': 0, 'data3_2d': 1, 'data4_2d': 2 }
f = open("ca_index_2.txt","r")
for line in f.readlines():
    toks = line.split(' ')
    x = int(toks[0])
    y = int(toks[1])
    
    r = parse.parse("{}/{}/{}", toks[2])
    dataset = datasets[r[0]]
    ruleidx = int(r[1])
    
    if dataset == 0:
        colors = t1colors[ruleidx]
    elif dataset == 1:
        colors = t2colors[ruleidx]
    elif dataset == 2:
        colors = t3colors[ruleidx]
        
    index['%d_%d' % (x,y)] = [dataset, ruleidx, colors]

f.close()
    
net = Triplet().to(DEVICE)
net.load_state_dict(torch.load("triplet_ca_2d_multitype.pth", map_location=torch.device(DEVICE)))
net.eval()

#clf = pickle.load(open("umap_classifier_multi_2.pkl","rb"))

Triplet(
  (l1a): Conv3d(3, 32, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2))
  (b1a): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (l1b): Conv3d(32, 32, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2))
  (b1b): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (p1): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (l2a): Conv3d(32, 64, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2))
  (b2a): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (l2b): Conv3d(64, 64, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2))
  (b2b): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (p2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (l3a): Conv3d(64, 128, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2))
  (b3a): BatchNorm3d(128, eps=1e-05, momentum=0.1

In [6]:
def getCA(x,y):
    xi = x//32
    yi = y//32
    addr = "%d_%d" % (xi,yi)
    
    if addr in index:
        ca_info = index[addr]
        ca = CA(256, 256)
        ca.set_rule(ca_info[0], ca_info[1], ca_info[2])
        ca.init_state()
        
        print(ca_info[0], ca_info[1], ca_info[2])
        return ca
    else:
        return None

In [7]:
pygame.init()
screen = pygame.display.set_mode((1000, 1000), 0, 24)

In [8]:
window_x = 0
window_y = 0

simulator = None

stop = False
delay = 0
frameskip = 1
frame = 0
s_image = None

while not stop:
    events = pygame.event.get()
    if pygame.mouse.get_focused():
        x,y = pygame.mouse.get_pos()
             
        click = pygame.mouse.get_pressed()
        keys = pygame.key.get_pressed()
        
        if click[2] and delay<=0:
            window_x = window_x + (x-500)
            window_y = window_y + (y-500)
            
            if window_x<0:
                window_x=0
            if window_y<0:
                window_y=0
                
            if window_x>4096-1000:
                window_x=4096-1000
            if window_y>4096-1000:
                window_y=4096-1000
                
            delay = 10
        
        if click[0] and delay<=0:
            simulator = getCA(window_y + y, window_x + x)
            delay = 10
        
        if keys[pygame.K_q]:
            stop = True
            
        if keys[pygame.K_KP_PLUS] and delay<=0:
            frameskip += 1
            delay = 10
            print("Frameskip %d" % frameskip)
            
        if keys[pygame.K_KP_MINUS] and delay<=0:
            frameskip -= 1
            if frameskip<1:
                frameskip = 1
            delay = 10
            print("Frameskip %d" % frameskip)
            
        #print(x)
        #print(y)
        #print(click)
        #print(keys)
    
    if delay>0:
        delay -= 1
            
    screen.blit(im, (-window_x, -window_y))
    
    if simulator is not None:
        simulator.step()
        
        if frame%frameskip == 0 or s_image is None:
            s_image = simulator.render()
            s_image = pygame.pixelcopy.make_surface(s_image)
        
        screen.blit(s_image, (1000-256, 1000-256))
        frame += 1
        
    pygame.display.update()   
    time.sleep(0.005)

1 1679 3
1 1679 3
2 1489 4
1 570 4
2 1669 5
1 744 2
1 1616 4
1 846 4
Frameskip 2
Frameskip 1
1 846 4
1 1616 4
Frameskip 2
1 934 5
Frameskip 1
1 1420 4
1 1554 5
2 1724 5
Frameskip 2
2 1724 5
Frameskip 1
1 1681 5
1 1408 4
1 829 3
1 1694 3
1 1261 3
1 1115 4
2 7 4
0 5009 2
0 5512 2
1 1278 4
1 1345 5
2 1346 5
1 1125 2
1 1125 2
2 113 3
2 1019 5
2 286 4
2 978 2
2 1094 2
2 1496 3
2 736 5
1 47 3
2 1470 2
2 482 5
Frameskip 2
2 1914 3
2 1926 3
2 920 2
2 1982 2
1 599 2
2 1861 4
1 505 5
2 1599 2
2 1252 3
2 1025 2
2 1157 5
2 1804 3
2 236 5
2 1360 3
2 1100 3
2 37 5
2 37 5
2 37 5
2 1387 3
2 914 4
2 980 4
2 1982 2
2 758 3
2 1985 4
2 1809 3
2 1091 5
1 174 4
2 1232 3
2 363 3
0 4008 4
1 1117 3
0 4644 2
0 5137 2
0 5772 2
2 1977 4
0 4930 2


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:94: RuntimeWarning: overflow encountered in long_scalars


0 5036 5
1 1117 3
0 5897 4
0 5158 3
2 31 2
1 792 3
0 5420 4
1 242 5
1 1586 5
2 1260 5
0 4250 2
2 349 4
2 1275 5
2 1275 5
1 1541 4
1 1018 4
Frameskip 1
Frameskip 1
Frameskip 2
Frameskip 3
Frameskip 2
Frameskip 1
Frameskip 2
2 1592 5
1 1147 5
Frameskip 3
1 651 3
1 1035 5
1 651 3
1 168 4
1 1250 3
2 1345 3
2 1165 4
1 1343 3
2 1487 4
Frameskip 2
Frameskip 1
Frameskip 2
Frameskip 3
1 805 4
1 977 5
Frameskip 2
Frameskip 1
Frameskip 2
Frameskip 3
1 939 4
1 887 3
0 1842 3
Frameskip 2
Frameskip 1
Frameskip 1
Frameskip 1
Frameskip 2
0 3456 2
0 390 2
0 757 2
0 2296 2
0 2646 2
0 454 3
0 2833 2
0 3090 3
0 1750 3
0 2691 4
0 1676 5
0 2021 3
0 2021 3
0 3186 3
0 3186 3
0 2047 5
0 2660 2
0 938 2
0 2691 4
0 2237 4
0 139 4
0 2865 5
0 2409 2
0 966 2
Frameskip 1
Frameskip 1
Frameskip 2
Frameskip 3
Frameskip 2
0 3374 3
0 2463 2
0 2225 5
Frameskip 1
Frameskip 2
Frameskip 3
0 2978 4
0 2701 3
0 1769 3
0 663 3
2 419 3
0 2701 3
0 2701 3
0 2701 3
0 5673 3
0 4394 4
Frameskip 2
Frameskip 1
Frameskip 2
0 2122 3
0 1326

In [73]:
print(simulator)

None


In [9]:
pygame.display.quit()

In [9]:
# 0 5268 5

In [15]:
simulator = CA(256, 256)
simulator.set_rule(0, 5268, 5)
#simulator.set_rule(2, 1380, 4)
simulator.init_state()

window_x = 0
window_y = 0

stop = False
delay = 0
frameskip = 1
frame = 0
s_image = None

while not stop:
    events = pygame.event.get()
    if pygame.mouse.get_focused():
        x,y = pygame.mouse.get_pos()
             
        click = pygame.mouse.get_pressed()
        keys = pygame.key.get_pressed()
        
        if keys[pygame.K_q]:
            stop = True
        
        if keys[pygame.K_r]:
            simulator.init_state()

        if keys[pygame.K_KP_PLUS] and delay<=0:
            frameskip += 1
            delay = 10
            print("Frameskip %d" % frameskip)
            
        if keys[pygame.K_KP_MINUS] and delay<=0:
            frameskip -= 1
            if frameskip<1:
                frameskip = 1
            delay = 10
            print("Frameskip %d" % frameskip)
                
    if delay>0:
        delay -= 1
            
    #screen.blit(im, (-window_x, -window_y))
    
    if simulator is not None:
        simulator.step()
        
        if frame%frameskip == 0 or s_image is None:
            s_image = simulator.render()
            s_image = pygame.pixelcopy.make_surface(s_image)
        
        screen.blit(s_image, (0,0))
        frame += 1
        
    pygame.display.update()   
    time.sleep(0.005)

Frameskip 2
Frameskip 1
Frameskip 2
